In [5]:
import plotly.graph_objects as go
import pandas as pd


# Parameters 2025, Vaud
H = 380_000
p_g = 0.30
p_hp = 0.17
L_g = 20
L_hp = 20
cu_g = 2.7     # kg per gas boiler
cu_hp = 36     # kg per heat pump
rr = 0.61      # end-of-life recycling rate
a = 0.80       # allocation of recovered to local manufacturing
m_loss = 0.05  # manufacturing loss
installs_per_year = H*(p_g+p_hp) / L_g   # replace gb + hp with only hp



In [6]:
copper_extraction_GHG = 3.5     # tCO2/tCu
copper_recycling_GHG = 0.3      # tCO2/tCu
electricity_demand_gb_production = 480 # kWh
average_electricity_demand_gb = 0.00264 # (15kW boiler)
emissions_to_air_gb = 0.056 # (15 kW boiler)


In [7]:

labels = [
    "Imports",       # 0
    "GB Stock",      # 1
    "Cu",            # 2
    "HP Stock",      # 3
    "Outflow"        # 4
]

# Example flows 
flow_imports_to_cu = 50
flow_gb_to_cu = 20
flow_cu_to_hp = 60
flow_hp_to_cu = 15      # circular recovered flow
flow_hp_to_outflow = 10 # unrecoverable flow

sources = [
    0,  # Imports -> Cu
    1,  # GB Stock -> Cu
    2,  # Cu -> HP Stock
    3,  # HP Stock -> Cu (circular flow)
    3   # HP Stock -> Outflow
]

targets = [
    2,  # Cu
    2,  # Cu
    3,  # HP Stock
    2,  # Cu (circularity)
    4   # Outflow
]

values = [
    flow_imports_to_cu,
    flow_gb_to_cu,
    flow_cu_to_hp,
    flow_hp_to_cu,
    flow_hp_to_outflow
]

colors = [
    "rgba(0,100,255,0.6)",   # Imports -> Cu
    "rgba(0,150,200,0.6)",   # GB -> Cu
    "rgba(0,200,120,0.6)",   # Cu -> HP
    "rgba(0,180,80,0.6)",    # HP -> Cu (recycled)
    "rgba(255,120,50,0.6)"   # HP -> Outflow
]

# -------------------------------------------------
# Fixed positions for 4 layers 
node_x = [0.05, 0.05, 0.35, 0.65, 0.90]
node_y = [0.2, 0.7, 0.45, 0.45, 0.45]

fig = go.Figure(data=[go.Sankey(
    arrangement="fixed",
    node=dict(
        pad=20,
        thickness=22,
        line=dict(color="black", width=0.5),
        label=labels,
        x=node_x,
        y=node_y,
        color="rgba(80,80,80,0.4)"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=colors
    )
)])

fig.update_layout(
    title_text="Copper Sankey for Heating Transition: Imports → Cu → HP Stock → Outflow (with Circularity)",
    font_size=14
)

fig.show()


In [45]:
# GB copper stock: 11400 GB * 2.7kg = 307080 kg Cu = 307.8 tonn 
# # HP copper stock: 64 600 HP * 36 kg = 2 325.6 tonn 
# # HP stock is growing so increase each year is 
# # GB replacementrate + HP replacementrate + growing rate = 
# 4560 + 5000 + 440 = 10000 * 36kg = 360000 kg = 36 tonn 
# # Recycled GB = 0.61 * 4560 * 2.7kg # Recycled HP = 0.61 * 5000 * 36 kg 
# # Rest is outflow (waste)# Model values (tonnes)
rr = 0.61 
hp_cu_out = 5000 * 36/1000      # t
gb_cu_out = 4560 * 2.7/1000     # t
population_growth_cu = 1000 * 36/1000

hp_to_cu = rr * hp_cu_out
gb_to_cu = rr * gb_cu_out

imports_to_cu = hp_cu_out + gb_cu_out + population_growth_cu - hp_to_cu - gb_to_cu

gb_to_outflow = gb_cu_out - gb_to_cu
hp_to_outflow = hp_cu_out - hp_to_cu

cu_to_hp = imports_to_cu + gb_to_cu + hp_to_cu
# -------------------------

labels = [
    "Imports",   # 0
    "Cu Pool",   # 1
    "GB Stock (307.8 tonn)",  # 2
    "HP Stock (2325.6 tonn)",  # 3
    "Outflow"    # 4
]

sources = [
    0,  # Imports → Cu
    2,  # GB → Cu
    2,  # GB → Outflow
    1,  # Cu → HP
    3,  # HP → Cu
    3   # HP → Outflow
]

targets = [
    1,  # Imports → Cu
    1,  # GB → Cu
    4,  # GB → Outflow
    3,  # Cu → HP
    1,  # HP → Cu
    4   # HP → Outflow
]

values = [
    imports_to_cu,
    gb_to_cu,
    gb_to_outflow,
    cu_to_hp,
    hp_to_cu,
    hp_to_outflow
]

colors = [
    "rgba(0,100,255,0.6)",  # Imports → Cu
    "rgba(0,150,200,0.6)",  # GB → Cu
    "rgba(255,120,50,0.6)", # GB → Outflow
    "rgba(0,200,120,0.6)",  # Cu → HP
    "rgba(0,180,80,0.6)",   # HP → Cu
    "rgba(255,120,50,0.6)"  # HP → Outflow
]

# CLEAN LAYOUT
node_x = [0.05, 0.35, 0.65, 0.65, 0.93]
node_y = [0.50, 0.50, 0.05, 0.75, 0.50]
# (GB above, HP below)

fig = go.Figure(data=[go.Sankey(
    arrangement="fixed",
    node=dict(
        pad=20,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        x=node_x,
        y=node_y,
        color="rgba(80,80,80,0.4)"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=colors,
        customdata=values,
        hovertemplate='Flow: %{customdata:.2f} t<extra></extra>'
    )
)])

# Add flow labels at approximate midpoints
annotations = [
    dict(x=0.20, y=0.50, text=f"{imports_to_cu:.2f} t ->", showarrow=False, font=dict(size=12, color="black")),
    dict(x=0.55, y=0.85, text=f"<- {gb_to_cu:.2f} t", showarrow=False, font=dict(size=12, color="black")),
    dict(x=0.79, y=0.85, text=f"{gb_to_outflow:.2f} t ->", showarrow=False, font=dict(size=12, color="black")),
    dict(x=0.50, y=0.35, text=f"{cu_to_hp:.2f} t ->", showarrow=False, font=dict(size=12, color="black")),
    dict(x=0.50, y=-0.15, text=f" <- {hp_to_cu:.2f} t", showarrow=False, font=dict(size=12, color="black")),
    dict(x=0.93, y=0.45, text=f"{hp_to_outflow:.2f} t ->", showarrow=False, font=dict(size=12, color="black"))
]

fig.update_layout(
    title_text="Copper Flow Sankey Canton Vaud (2025)",
    font_size=14,
    annotations=annotations
)

fig.show()


In [ ]:
import plotly.graph_objects as go

# FLOW VALUES (t)
rr = 0.61
hp_cu_out = 5000 * 36/1000      # t
hp_to_cu = rr * hp_cu_out
hp_to_outflow = hp_cu_out - hp_to_cu
imports_to_cu = hp_cu_out - hp_to_cu
cu_to_hp = imports_to_cu + hp_to_cu

labels = ["Imports", "Cu Pool", "HP Stock", "Outflow"]

sources = [0, 1, 2, 2]    # Imports->Cu, Cu->HP, HP->Cu, HP->Outflow
targets = [1, 2, 1, 3]
values = [imports_to_cu, cu_to_hp, hp_to_cu, hp_to_outflow]

colors = [
    "rgba(0,100,255,0.6)",  # Imports → Cu
    "rgba(0,200,120,0.6)",  # Cu → HP
    "rgba(0,180,80,0.6)",   # HP → Cu
    "rgba(255,120,50,0.6)"  # HP → Outflow
]

# Node positions
node_x = [0.05, 0.35, 0.65, 0.93]
node_y = [0.40, 0.40, 0.65, 0.30]

fig = go.Figure(go.Sankey(
    arrangement="fixed",
    node=dict(
        pad=20,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        x=node_x,
        y=node_y,
        color="rgba(80,80,80,0.4)"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=colors,
        customdata=values,
        hovertemplate='Flow: %{customdata:.2f} t<extra></extra>'
    )
))

# Add annotations for flows
annotations = [
    dict(x=0.20, y=0.70, text=f"{imports_to_cu:.2f} t ->", showarrow=False, font=dict(size=12, color="black")),
    dict(x=0.50, y=0.55, text=f"{cu_to_hp:.2f} t ->", showarrow=False, font=dict(size=12, color="black")),
    dict(x=0.49, y=-0.05, text=f" <- {hp_to_cu:.2f} t", showarrow=False, font=dict(size=12, color="black")),
    dict(x=0.83, y=0.60, text=f"{hp_to_outflow:.2f} t ->", showarrow=False, font=dict(size=12, color="black"))
]

fig.update_layout(
    title_text="Copper Flow Sankey Canton Vaud (2050)",
    font_size=14,
    annotations=annotations
)

fig.show()
